In [10]:
import pandas as pd
import numpy as np

cells = pd.read_csv('data.csv')
cells.head()


,Well,Field,CellNumber,Top,Left,XCentroid,YCentroid,ObjectTotalIntenCh1,ObjectAvgIntenCh1,TotalIntenCh2,AvgIntenCh2,TotalIntenCh3,AvgIntenCh3
0,B2,1.0,1,54,706,719,70,323333,644.089641,13947,27.782869,131767,262.484064
1,B2,1.0,2,75,698,711,88,445511,1060.740476,24663,58.721429,753451,1793.930952
2,B2,1.0,3,89,689,703,105,329831,618.819887,16128,30.258912,154356,289.598499
3,B2,1.0,4,96,678,683,100,52911,594.505618,4739,53.247191,4733,53.179775
4,B2,1.0,5,129,738,745,134,46860,593.164557,3291,41.658228,3766,47.670886


In [11]:
pi = cells[['Well','ObjectAvgIntenCh1', 'AvgIntenCh3']].copy()
pi = pi.rename(columns={"Well": "Well",
                        "ObjectAvgIntenCh1": "h_intensity",
                        "AvgIntenCh3": "pi_intensity"})
pi.head()


,Well,h_intensity,pi_intensity
0,B2,644.089641,262.484064
1,B2,1060.740476,1793.930952
2,B2,618.819887,289.598499
3,B2,594.505618,53.179775
4,B2,593.164557,47.670886


In [12]:
negativeControl = pi.loc[(pi['Well'] == 'B11') |(pi['Well'] == 'C11')| (pi['Well'] == 'D11') | (pi['Well'] == 'E2') | (pi['Well'] == 'F2') | (pi['Well'] == 'G2')]
negativeControl.std()
threshold = 3*negativeControl.pi_intensity.std()
print(threshold)

617.3261022178222


/Users/jimendi1/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


In [13]:
positiveControl = cells.loc[(cells['Well']=='B2') | (cells['Well']=='C2')| (cells['Well']=='D2')|(cells['Well']=='E11') |(cells['Well']=='F11')|(cells['Well']=='G11')]
pos_ctrl_int_pi = positiveControl['AvgIntenCh3']

In [14]:
import scipy.stats as st
threshold_ttest = st.ttest_1samp(pos_ctrl_int_pi, threshold)
if(threshold_ttest.pvalue > 0.05):
    raise Exception("too much overlap between negative and postive controls' intensities")

In [15]:
default = ['alive' for x in range(len(pi))]
pi['State'] = default
pi.loc[(pi['pi_intensity'] >= threshold), 'State'] = 'dead'
pi.head()

,Well,h_intensity,pi_intensity,State
0,B2,644.089641,262.484064,alive
1,B2,1060.740476,1793.930952,dead
2,B2,618.819887,289.598499,alive
3,B2,594.505618,53.179775,alive
4,B2,593.164557,47.670886,alive
